In [5]:
import json
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, ChiSqSelector, IDF, Normalizer
from pyspark.sql import SparkSession
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest


In [6]:
# Read the JSON file
file = open("./reviews_devset copy.json")
reviews_devset = file.readlines()
reviewsDF = pd.DataFrame.from_records(list(map(json.loads, reviews_devset)))

# Categorize the labels
reviewsDF["category"] = pd.Categorical(reviewsDF["category"])
reviewsDF["labels"] = reviewsDF["category"].cat.codes


In [7]:
# Create a SparkSession
spark = SparkSession.builder.master("local[1]").appName("part3").getOrCreate()

# Create a Spark DataFrame
spark_dff = spark.createDataFrame(reviewsDF.loc[0:1000, ["reviewText", "category", "labels"]])

In [9]:
# Define the pipeline stages
tokenizer = Tokenizer(inputCol="reviewText", outputCol="rawWords")

remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words", caseSensitive=False)

hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol="rawFeatures", numFeatures=2000)

idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")

selector = ChiSqSelector(numTopFeatures=2000, featuresCol=idf.getOutputCol(), outputCol="selectedFeatures", labelCol="labels")

normalizer = Normalizer(inputCol="selectedFeatures", outputCol="normalizedFeatures", p=2.0)

svc = LinearSVC(maxIter=10, regParam=0.1)

classifier = OneVsRest(classifier=svc, labelCol="labels", featuresCol="normalizedFeatures", predictionCol="prediction")

#svc = LinearSVC(maxIter=10, regParam=0.1)

classifier = OneVsRest(classifier=classifier, labelCol="labels", featuresCol="normalizedFeatures", predictionCol="prediction")

# Create the pipeline
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, selector, normalizer, classifier])





In [10]:
# Split the data into training, validation, and test sets
(trainingData, validationData, testData) = spark_dff.randomSplit([0.6, 0.2, 0.2], seed=42)

# Fit the pipeline on the training data
model = pipeline.fit(trainingData)

Py4JJavaError: An error occurred while calling o82.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (128.130.249.198 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.s

In [11]:
# Make predictions on the validation data
predictions = model.transform(validationData)

# Evaluate the model using F1 measure
evaluator = MulticlassClassificationEvaluator(labelCol="labels", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)

print("F1 Score: ", f1_score)


NameError: name 'model' is not defined